In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.api as sm
import scipy
from scipy.stats import norm
from scipy.stats.mstats import gmean

from statsmodels.nonparametric.smoothers_lowess import lowess

from numpy.lib.stride_tricks import sliding_window_view
import functions_mem as fm

from datetime import datetime
from pathlib import Path

In [2]:
import importlib
import functions_mem
importlib.reload(functions_mem)



<module 'functions_mem' from '/Users/julianeoliveira/Documents/github/IDEA_model/tmp/functions_mem.py'>

In [9]:
#dir(functions_mem)


# Load data

In [3]:
df = pd.read_parquet("/Users/julianeoliveira/Downloads/aesop_2026_01_09_mun.parquet")

In [4]:
lst = ['co_uf','nm_uf','nm_municipio','co_ibge', 'epiyear', 'epiweek','year_week', 'atend_totais', 'atend_ivas','ra_atend_ivas','ra_atend_ivas_ma',]

df = df[lst]

# Start run method

### Test the code for one city (this part of the code will is commented when running the python file)

In [6]:
res = fm.find_best_season_and_delta(
    df,
    110001,
    delta_start=0.02,
    delta_max=0.04,
    delta_step=0.001,
    k_start_min=10,
    r_max=20,
)

In [7]:
print(res)


   epiyear  r_j_estr  k_start  k_end  co_ibge  season_def  delta_used  co_uf  \
0     2017        27        2     28   110001  season_w39       0.022     11   
1     2018        13       35     47   110001  season_w39       0.022     11   
2     2019        15       36     50   110001  season_w39       0.022     11   
3     2020        12       41     52   110001  season_w39       0.022     11   
4     2021        18       17     34   110001  season_w39       0.022     11   
5     2022        15        2     16   110001  season_w39       0.022     11   
6     2023         8       11     18   110001  season_w39       0.022     11   
7     2024         9       28     36   110001  season_w39       0.022     11   
8     2025        14       31     44   110001  season_w39       0.022     11   
9     2026         6       10     15   110001  season_w39       0.022     11   

      nm_uf  
0  Rondônia  
1  Rondônia  
2  Rondônia  
3  Rondônia  
4  Rondônia  
5  Rondônia  
6  Rondônia  
7  Rond

### Run for all cities

In [ ]:
results = []

for city in df.co_ibge.unique():
    print(f"Processing {city}...")
    res = find_best_season_and_delta(df, city)
    if res is not None:
        results.append(res)

final_selected = pd.concat(results, ignore_index=True)


# Save output to parquet


In [ ]:
out_dir = Path("/Users/julianeoliveira/Downloads")

fname = f"def_sea_MEM_out_{datetime.now():%d_%m_%Y}.parquet"

final_selected.to_parquet(out_dir / fname)


In [ ]:
set_bad =  set(df.co_ibge.unique()) - set(final_selected.co_ibge.unique())
dta = pd.DataFrame({"co_ibge": sorted(set_bad)})

fname2 = f"set_bad_cities_{datetime.now():%d_%m_%Y}.csv"

dta.to_csv(out_dir / fname2, index=False)

In [ ]:
#final_selected.to_parquet('/Users/julianeoliveira/Downloads/def_sea_MEM_23_01_2026.parquet')

In [30]:
final_selected.co_ibge.nunique()

5249